# Video Search App

## Prerequisites

### Install Dependencies

In [2]:
# !pip install -q --upgrade google-cloud-aiplatform 
!pip install -q --upgrade google-cloud-dlp python-docx --upgrade google-auth
!pip install -q --upgrade google-cloud-videointelligence
!pip install -q bigframes==0.26
!pip install -q --upgrade moviepy
!pip install -q unidecode

!pip install -q --upgrade pytube
# !pip install -q --upgrade google-cloud-discoveryengine
!pip install -q langchain_google_vertexai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-documentai-toolbox 0.13.0a0 requires grpc-google-iam-v1<0.13dev,>=0.12.6, but you have grpc-google-iam-v1 0.13.0 which is incompatible.
google-cloud-translate 2.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.15.0, but you have google-api-core 2.19.1 which is incompatible.
google-cloud-translate 2.0.1 requires google-cloud-core<2.0dev,>=1.1.0, but you have google-cloud-core 2.4.1 which is incompatible.
kfp 2.4.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 29.0.0 which is incompatible.
kfp 2.4.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
kfp 2.4.0 requires urllib3<2.0.0, but you have urllib3 2.2.1 which is incompatible.
langchain-google-genai 1.0.3 requires google-generativeai<0.6.0,>=0.5.2, but you have google-generativeai 0.7.1 which 

### Setup Environment Variables

In [3]:
from langchain_google_vertexai import VertexAIEmbeddings


In [4]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

UID = UNIQUE_PREFIX

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}

!mkdir content
!rm -r content/clips/
!mkdir content/clips/
!rm -r content/frames/
!mkdir content/frames/

Updated property [core/project].
Creating gs://my-project-0004-346516-pytorch112kagglewbi-us-central1/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.
mkdir: cannot create directory ‘content’: File exists


### Initialize Libraries

In [5]:
import sys
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import Image as vision_model_Image
from vertexai.vision_models import MultiModalEmbeddingModel
import glob
import os
import time
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union
from IPython.display import display
import PIL
import numpy as np
import pandas as pd
import requests
from vertexai.generative_models import (
    GenerationConfig,
    HarmBlockThreshold,
    HarmCategory,
    Image,
)
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import Image as vision_model_Image
from vertexai.vision_models import MultiModalEmbeddingModel
from moviepy.editor import VideoFileClip
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine

### Download and upload videos to a bucket

#### Helper Functions

In [6]:
import pytube
from google.cloud import storage
import unidecode

# -1.1) Fetching from youtube
def download_video(video_url):
    # Create a YouTube object
    yt = YouTube(video_url)
    
    # Filter to get the highest resolution stream that includes both video and audio
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    
    # Download the video
    if stream:
        print(f"Going to download {video_url}")
        stream.download()
        print(f"Downloaded into {stream.default_filename}")
    else:
        print("No suitable stream found")
    return stream.default_filename


def upload_file_to_gcs(bucket_name, source_file_path, destination_blob_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)
    return blob

def upload_file_to_bucket(bucket_name, source_file_path_name): 
    source_file_path = f"./{source_file_path_name}"
    destination_blob_name = source_file_path_name.replace(' ', '_')
    destination_blob_name = unidecode.unidecode(destination_blob_name)
    uploaded_blob         = upload_file_to_gcs(bucket_name, source_file_path, destination_blob_name)
    print(f"File [{source_file_path}] \nUploaded to [{uploaded_blob.name}] \nIn bucket [{bucket_name}]")
    return destination_blob_name

### Download and upload Videos from youtube

In [7]:
# Define the video URL
video_url = "https://www.youtube.com/watch?v=ZBRLpvgcTnM"

# Commented out because Pytube is down (as of Jul 20, 2024). Feel free to uncomment if Pytube is working again
# downloaded_video = download_video(video_url)

#Comment out below code if Pytube is working:
#Change video to your video in content folder
downloaded_video = "./content/googleio.mp4"

### Create csv file

In [8]:
import csv

with open('shots.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = ["Id", "start_time", "end_time", "clip_name", "frame_name", "associated_text", "associated_speech", "associated_object", "description"]
    writer.writerow(field)

## Process videos

### Load AI Model

#### Helper Functions

In [9]:
# Functions for getting text and image embeddings

def get_text_embedding_from_text_embedding_model(
    text: str,
    return_array: Optional[bool] = False,
) -> list:
    """
    Generates a numerical text embedding from a provided text input using a text embedding model.

    Args:
        text: The input text string to be embedded.
        return_array: If True, returns the embedding as a NumPy array.
                      If False, returns the embedding as a list. (Default: False)

    Returns:
        list or numpy.ndarray: A 768-dimensional vector representation of the input text.
                               The format (list or NumPy array) depends on the
                               value of the 'return_array' parameter.
    """
    embeddings = text_embedding_model.get_embeddings([text])
    text_embedding = [embedding.values for embedding in embeddings][0]

    if return_array:
        text_embedding = np.fromiter(text_embedding, dtype=float)

    # returns 768 dimensional array
    return text_embedding


def get_image_embedding_from_multimodal_embedding_model(
    image_uri: str,
    embedding_size: int = 512,
    text: Optional[str] = None,
    return_array: Optional[bool] = False,
) -> list:
    """Extracts an image embedding from a multimodal embedding model.
    The function can optionally utilize contextual text to refine the embedding.

    Args:
        image_uri (str): The URI (Uniform Resource Identifier) of the image to process.
        text (Optional[str]): Optional contextual text to guide the embedding generation. Defaults to "".
        embedding_size (int): The desired dimensionality of the output embedding. Defaults to 512.
        return_array (Optional[bool]): If True, returns the embedding as a NumPy array.
        Otherwise, returns a list. Defaults to False.

    Returns:
        list: A list containing the image embedding values. If `return_array` is True, returns a NumPy array instead.
    """
    # image = Image.load_from_file(image_uri)
    image = vision_model_Image.load_from_file(image_uri)
    embeddings = multimodal_embedding_model.get_embeddings(
        image=image, contextual_text=text, dimension=embedding_size
    )
    image_embedding = embeddings.image_embedding

    if return_array:
        image_embedding = np.fromiter(image_embedding, dtype=float)

    return image_embedding


def load_image_bytes(image_path):
    """Loads an image from a URL or local file path.

    Args:
        image_uri (str): URL or local file path to the image.

    Raises:
        ValueError: If `image_uri` is not provided.

    Returns:
        bytes: Image bytes.
    """
    # Check if the image_uri is provided
    if not image_path:
        raise ValueError("image_uri must be provided.")

    # Load the image from a weblink
    if image_path.startswith("http://") or image_path.startswith("https://"):
        response = requests.get(image_path, stream=True)
        if response.status_code == 200:
            return response.content

    # Load the image from a local path
    else:
        return open(image_path, "rb").read()





# Add colors to the print
class Color:
    """
    This class defines a set of color codes that can be used to print text in different colors.
    This will be used later to print citations and results to make outputs more readable.
    """

    PURPLE: str = "\033[95m"
    CYAN: str = "\033[96m"
    DARKCYAN: str = "\033[36m"
    BLUE: str = "\033[94m"
    GREEN: str = "\033[92m"
    YELLOW: str = "\033[93m"
    RED: str = "\033[91m"
    BOLD: str = "\033[1m"
    UNDERLINE: str = "\033[4m"
    END: str = "\033[0m"


def get_text_overlapping_chunk(
    text: str, character_limit: int = 1000, overlap: int = 100
) -> dict:
    """
    * Breaks a text document into chunks of a specified size, with an overlap between chunks to preserve context.
    * Takes a text document, character limit per chunk, and overlap between chunks as input.
    * Returns a dictionary where the keys are chunk numbers and the values are the corresponding text chunks.

    Args:
        text: The text document to be chunked.
        character_limit: Maximum characters per chunk (defaults to 1000).
        overlap: Number of overlapping characters between chunks (defaults to 100).

    Returns:
        A dictionary where keys are chunk numbers and values are the corresponding text chunks.

    Raises:
        ValueError: If `overlap` is greater than `character_limit`.

    """

    if overlap > character_limit:
        raise ValueError("Overlap cannot be larger than character limit.")

    # Initialize variables
    chunk_number = 1
    chunked_text_dict = {}

    # Iterate over text with the given limit and overlap
    for i in range(0, len(text), character_limit - overlap):
        end_index = min(i + character_limit, len(text))
        chunk = text[i:end_index]

        # Encode and decode for consistent encoding
        chunked_text_dict[chunk_number] = chunk.encode("ascii", "ignore").decode(
            "utf-8", "ignore"
        )

        # Increment chunk number
        chunk_number += 1

    return chunked_text_dict


def get_page_text_embedding(text_data: Union[dict, str]) -> dict:
    """
    * Generates embeddings for each text chunk using a specified embedding model.
    * Takes a dictionary of text chunks and an embedding size as input.
    * Returns a dictionary where the keys are chunk numbers and the values are the corresponding embeddings.

    Args:
        text_data: Either a dictionary of pre-chunked text or the entire page text.
        embedding_size: Size of the embedding vector (defaults to 128).

    Returns:
        A dictionary where keys are chunk numbers or "text_embedding" and values are the corresponding embeddings.

    """

    embeddings_dict = {}

    if isinstance(text_data, dict):
        # Process each chunk
        # print(text_data)
        for chunk_number, chunk_value in text_data.items():
            text_embd = get_text_embedding_from_text_embedding_model(text=chunk_value)
            embeddings_dict[chunk_number] = text_embd
    else:
        # Process the first 1000 characters of the page text
        text_embd = get_text_embedding_from_text_embedding_model(text=text_data)
        embeddings_dict["text_embedding"] = text_embd

    return embeddings_dict



def get_gemini_response(
    generative_multimodal_model,
    model_input: List[str],
    stream: bool = True,
    generation_config: Optional[GenerationConfig] = GenerationConfig(
        temperature=0.2, max_output_tokens=2048
    ),
    safety_settings: Optional[dict] = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
) -> str:
    """
    This function generates text in response to a list of model inputs.

    Args:
        model_input: A list of strings representing the inputs to the model.
        stream: Whether to generate the response in a streaming fashion (returning chunks of text at a time) or all at once. Defaults to False.

    Returns:
        The generated text as a string.
    """
    response = generative_multimodal_model.generate_content(
        model_input,
        generation_config=generation_config,
        stream=stream,
        safety_settings=safety_settings,
    )
    response_list = []

    for chunk in response:
        try:
            response_list.append(chunk.text)
        except Exception as e:
            print(
                "Exception occurred while calling gemini. Something is wrong. Lower the safety thresholds [safety_settings: BLOCK_NONE ] if not already done. -----",
                e,
            )
            response_list.append("Exception occurred")
            continue
    response = "".join(response_list)

    return response


def get_text_metadata_df(
    filename: str, text_metadata: Dict[Union[int, str], Dict]
) -> pd.DataFrame:
    """
    This function takes a filename and a text metadata dictionary as input,
    iterates over the text metadata dictionary and extracts the text, chunk text,
    and chunk embeddings for each page, creates a Pandas DataFrame with the
    extracted data, and returns it.

    Args:
        filename: The filename of the document.
        text_metadata: A dictionary containing the text metadata for each page.

    Returns:
        A Pandas DataFrame with the extracted text, chunk text, and chunk embeddings for each page.
    """

    final_data_text: List[Dict] = []

    for key, values in text_metadata.items():
        for chunk_number, chunk_text in values["chunked_text_dict"].items():
            data: Dict = {}
            data["file_name"] = filename
            data["page_num"] = int(key) + 1
            data["text"] = values["text"]
            data["text_embedding_page"] = values["page_text_embeddings"][
                "text_embedding"
            ]
            data["chunk_number"] = chunk_number
            data["chunk_text"] = chunk_text
            data["text_embedding_chunk"] = values["chunk_embeddings_dict"][chunk_number]

            final_data_text.append(data)

    return_df = pd.DataFrame(final_data_text)
    return_df = return_df.reset_index(drop=True)
    return return_df


def get_image_metadata_df(
    filename: str, image_metadata: Dict[Union[int, str], Dict]
) -> pd.DataFrame:
    """
    This function takes a filename and an image metadata dictionary as input,
    iterates over the image metadata dictionary and extracts the image path,
    image description, and image embeddings for each image, creates a Pandas
    DataFrame with the extracted data, and returns it.

    Args:
        filename: The filename of the document.
        image_metadata: A dictionary containing the image metadata for each page.

    Returns:
        A Pandas DataFrame with the extracted image path, image description, and image embeddings for each image.
    """

    final_data_image: List[Dict] = []
    for key, values in image_metadata.items():
        for _, image_values in values.items():
            data: Dict = {}
            data["file_name"] = filename
            data["page_num"] = int(key) + 1
            data["img_num"] = int(image_values["img_num"])
            data["img_path"] = image_values["img_path"]
            data["img_desc"] = image_values["img_desc"]
            # data["mm_embedding_from_text_desc_and_img"] = image_values[
            #     "mm_embedding_from_text_desc_and_img"
            # ]
            data["mm_embedding_from_img_only"] = image_values[
                "mm_embedding_from_img_only"
            ]
            data["text_embedding_from_image_description"] = image_values[
                "text_embedding_from_image_description"
            ]
            final_data_image.append(data)

    return_df = pd.DataFrame(final_data_image).dropna()
    return_df = return_df.reset_index(drop=True)
    return return_df


def get_document_metadata(
    generative_multimodal_model,
    pdf_folder_path: str,
    image_save_dir: str,
    image_description_prompt: str,
    embedding_size: int = 128,
    generation_config: Optional[GenerationConfig] = GenerationConfig(
        temperature=0.2, max_output_tokens=2048
    ),
    safety_settings: Optional[dict] = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
    add_sleep_after_page: bool = False,
    sleep_time_after_page: int = 2,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    This function takes a PDF path, an image save directory, an image description prompt, an embedding size, and a text embedding text limit as input.

    Args:
        pdf_path: The path to the PDF document.
        image_save_dir: The directory where extracted images should be saved.
        image_description_prompt: A prompt to guide Gemini for generating image descriptions.
        embedding_size: The dimensionality of the embedding vectors.
        text_emb_text_limit: The maximum number of tokens for text embedding.

    Returns:
        A tuple containing two DataFrames:
            * One DataFrame containing the extracted text metadata for each page of the PDF, including the page text, chunked text dictionaries, and chunk embedding dictionaries.
            * Another DataFrame containing the extracted image metadata for each image in the PDF, including the image path, image description, image embeddings (with and without context), and image description text embedding.
    """

    text_metadata_df_final, image_metadata_df_final = pd.DataFrame(), pd.DataFrame()

    for pdf_path in glob.glob(pdf_folder_path + "/*.pdf"):
        print(
            "\n\n",
            "Processing the file: ---------------------------------",
            pdf_path,
            "\n\n",
        )

        doc, num_pages = get_pdf_doc_object(pdf_path)

        file_name = pdf_path.split("/")[-1]

        text_metadata: Dict[Union[int, str], Dict] = {}
        image_metadata: Dict[Union[int, str], Dict] = {}

        for page_num in range(num_pages):
            print(f"Processing page: {page_num + 1}")

            page = doc[page_num]

            text = page.get_text()
            (
                text,
                page_text_embeddings_dict,
                chunked_text_dict,
                chunk_embeddings_dict,
            ) = get_chunk_text_metadata(page, embedding_size=embedding_size)

            text_metadata[page_num] = {
                "text": text,
                "page_text_embeddings": page_text_embeddings_dict,
                "chunked_text_dict": chunked_text_dict,
                "chunk_embeddings_dict": chunk_embeddings_dict,
            }

            images = page.get_images()
            image_metadata[page_num] = {}

            for image_no, image in enumerate(images):
                image_number = int(image_no + 1)
                image_metadata[page_num][image_number] = {}

                image_for_gemini, image_name = get_image_for_gemini(
                    doc, image, image_no, image_save_dir, file_name, page_num
                )

                print(
                    f"Extracting image from page: {page_num + 1}, saved as: {image_name}"
                )

                response = get_gemini_response(
                    generative_multimodal_model,
                    model_input=[image_description_prompt, image_for_gemini],
                    generation_config=generation_config,
                    safety_settings=safety_settings,
                    stream=True,
                )

                image_embedding = get_image_embedding_from_multimodal_embedding_model(
                    image_uri=image_name,
                    embedding_size=embedding_size,
                )

                image_description_text_embedding = (
                    get_text_embedding_from_text_embedding_model(text=response)
                )

                image_metadata[page_num][image_number] = {
                    "img_num": image_number,
                    "img_path": image_name,
                    "img_desc": response,
                    # "mm_embedding_from_text_desc_and_img": image_embedding_with_description,
                    "mm_embedding_from_img_only": image_embedding,
                    "text_embedding_from_image_description": image_description_text_embedding,
                }

            # Add sleep to reduce issues with Quota error on API
            if add_sleep_after_page:
                time.sleep(sleep_time_after_page)
                print(
                    "Sleeping for ",
                    sleep_time_after_page,
                    """ sec before processing the next page to avoid quota issues. You can disable it: "add_sleep_after_page = False"  """,
                )

        text_metadata_df = get_text_metadata_df(file_name, text_metadata)
        image_metadata_df = get_image_metadata_df(file_name, image_metadata)

        text_metadata_df_final = pd.concat(
            [text_metadata_df_final, text_metadata_df], axis=0
        )
        image_metadata_df_final = pd.concat(
            [
                image_metadata_df_final,
                image_metadata_df.drop_duplicates(subset=["img_desc"]),
            ],
            axis=0,
        )

        text_metadata_df_final = text_metadata_df_final.reset_index(drop=True)
        image_metadata_df_final = image_metadata_df_final.reset_index(drop=True)

    return text_metadata_df_final, image_metadata_df_final


# Helper Functions


def get_user_query_text_embeddings(user_query: str) -> np.ndarray:
    """
    Extracts text embeddings for the user query using a text embedding model.

    Args:
        user_query: The user query text.
        embedding_size: The desired embedding size.

    Returns:
        A NumPy array representing the user query text embedding.
    """

    return get_text_embedding_from_text_embedding_model(user_query)


def get_user_query_image_embeddings(
    image_query_path: str, embedding_size: int
) -> np.ndarray:
    """
    Extracts image embeddings for the user query image using a multimodal embedding model.

    Args:
        image_query_path: The path to the user query image.
        embedding_size: The desired embedding size.

    Returns:
        A NumPy array representing the user query image embedding.
    """

    return get_image_embedding_from_multimodal_embedding_model(
        image_uri=image_query_path, embedding_size=embedding_size
    )


def get_cosine_score(
    dataframe: pd.DataFrame, column_name: str, input_text_embd: np.ndarray
) -> float:
    """
    Calculates the cosine similarity between the user query embedding and the dataframe embedding for a specific column.

    Args:
        dataframe: The pandas DataFrame containing the data to compare against.
        column_name: The name of the column containing the embeddings to compare with.
        input_text_embd: The NumPy array representing the user query embedding.

    Returns:
        The cosine similarity score (rounded to two decimal places) between the user query embedding and the dataframe embedding.
    """

    text_cosine_score = round(np.dot(dataframe[column_name], input_text_embd), 2)
    return text_cosine_score


def print_text_to_image_citation(
    final_images: Dict[int, Dict[str, Any]], print_top: bool = True
) -> None:
    """
    Prints a formatted citation for each matched image in a dictionary.

    Args:
        final_images: A dictionary containing information about matched images,
                    with keys as image number and values as dictionaries containing
                    image path, page number, page text, cosine similarity score, and image description.
        print_top: A boolean flag indicating whether to only print the first citation (True) or all citations (False).

    Returns:
        None (prints formatted citations to the console).
    """

    color = Color()

    # Iterate through the matched image citations
    for imageno, image_dict in final_images.items():
        # Print the citation header
        print(
            color.RED + f"Citation {imageno + 1}:",
            "Matched image path, page number and page text: \n" + color.END,
        )

        # Print the cosine similarity score
        print(color.BLUE + "score: " + color.END, image_dict["cosine_score"])

        # Print the file_name
        print(color.BLUE + "file_name: " + color.END, image_dict["file_name"])

        # Print the image path
        print(color.BLUE + "path: " + color.END, image_dict["img_path"])

        # Print the page number
        print(color.BLUE + "page number: " + color.END, image_dict["page_num"])

        # Print the page text
        print(
            color.BLUE + "page text: " + color.END, "\n".join(image_dict["page_text"])
        )

        # Print the image description
        print(
            color.BLUE + "image description: " + color.END,
            image_dict["image_description"],
        )

        # Only print the first citation if print_top is True
        if print_top and imageno == 0:
            break


def print_text_to_text_citation(
    final_text: Dict[int, Dict[str, Any]],
    print_top: bool = True,
    chunk_text: bool = True,
) -> None:
    """
    Prints a formatted citation for each matched text in a dictionary.

    Args:
        final_text: A dictionary containing information about matched text passages,
                    with keys as text number and values as dictionaries containing
                    page number, cosine similarity score, chunk number (optional),
                    chunk text (optional), and page text (optional).
        print_top: A boolean flag indicating whether to only print the first citation (True) or all citations (False).
        chunk_text: A boolean flag indicating whether to print individual text chunks (True) or the entire page text (False).

    Returns:
        None (prints formatted citations to the console).
    """

    color = Color()

    # Iterate through the matched text citations
    for textno, text_dict in final_text.items():
        # Print the citation header
        print(color.RED + f"Citation {textno + 1}:", "Matched text: \n" + color.END)

        # Print the cosine similarity score
        print(color.BLUE + "score: " + color.END, text_dict["cosine_score"])

        # Print the file_name
        print(color.BLUE + "file_name: " + color.END, text_dict["file_name"])

        # Print the page number
        print(color.BLUE + "page_number: " + color.END, text_dict["page_num"])

        # Print the matched text based on the chunk_text argument
        if chunk_text:
            # Print chunk number and chunk text
            print(color.BLUE + "chunk_number: " + color.END, text_dict["chunk_number"])
            print(color.BLUE + "chunk_text: " + color.END, text_dict["chunk_text"])
        else:
            # Print page text
            print(color.BLUE + "page text: " + color.END, text_dict["page_text"])

        # Only print the first citation if print_top is True
        if print_top and textno == 0:
            break


def get_similar_image_from_query(
    text_metadata_df: pd.DataFrame,
    image_metadata_df: pd.DataFrame,
    query: str = "",
    image_query_path: str = "",
    column_name: str = "",
    image_emb: bool = True,
    top_n: int = 3,
    embedding_size: int = 128,
) -> Dict[int, Dict[str, Any]]:
    """
    Finds the top N most similar images from a metadata DataFrame based on a text query or an image query.

    Args:
        text_metadata_df: A Pandas DataFrame containing text metadata associated with the images.
        image_metadata_df: A Pandas DataFrame containing image metadata (paths, descriptions, etc.).
        query: The text query used for finding similar images (if image_emb is False).
        image_query_path: The path to the image used for finding similar images (if image_emb is True).
        column_name: The column name in the image_metadata_df containing the image embeddings or captions.
        image_emb: Whether to use image embeddings (True) or text captions (False) for comparisons.
        top_n: The number of most similar images to return.
        embedding_size: The dimensionality of the image embeddings (only used if image_emb is True).

    Returns:
        A dictionary containing information about the top N most similar images, including cosine scores, image objects, paths, page numbers, text excerpts, and descriptions.
    """
    # Check if image embedding is used
    if image_emb:
        # Calculate cosine similarity between query image and metadata images
        user_query_image_embedding = get_user_query_image_embeddings(
            image_query_path, embedding_size
        )
        cosine_scores = image_metadata_df.apply(
            lambda x: get_cosine_score(x, column_name, user_query_image_embedding),
            axis=1,
        )
    else:
        # Calculate cosine similarity between query text and metadata image captions
        user_query_text_embedding = get_user_query_text_embeddings(query)
        cosine_scores = image_metadata_df.apply(
            lambda x: get_cosine_score(x, column_name, user_query_text_embedding),
            axis=1,
        )

    # Remove same image comparison score when user image is matched exactly with metadata image
    cosine_scores = cosine_scores[cosine_scores < 1.0]

    # Get top N cosine scores and their indices
    top_n_cosine_scores = cosine_scores.nlargest(top_n).index.tolist()
    top_n_cosine_values = cosine_scores.nlargest(top_n).values.tolist()

    # Create a dictionary to store matched images and their information
    final_images: Dict[int, Dict[str, Any]] = {}

    for matched_imageno, indexvalue in enumerate(top_n_cosine_scores):
        # Create a sub-dictionary for each matched image
        final_images[matched_imageno] = {}

        # Store cosine score
        final_images[matched_imageno]["cosine_score"] = top_n_cosine_values[
            matched_imageno
        ]

        # Load image from file
        final_images[matched_imageno]["image_object"] = Image.load_from_file(
            image_metadata_df.iloc[indexvalue]["img_path"]
        )

        # Add file name
        final_images[matched_imageno]["file_name"] = image_metadata_df.iloc[indexvalue][
            "file_name"
        ]

        # Store image path
        final_images[matched_imageno]["img_path"] = image_metadata_df.iloc[indexvalue][
            "img_path"
        ]

        # Store page number
        final_images[matched_imageno]["page_num"] = image_metadata_df.iloc[indexvalue][
            "page_num"
        ]

        final_images[matched_imageno]["page_text"] = np.unique(
            text_metadata_df[
                (
                    text_metadata_df["page_num"].isin(
                        [final_images[matched_imageno]["page_num"]]
                    )
                )
                & (
                    text_metadata_df["file_name"].isin(
                        [final_images[matched_imageno]["file_name"]]
                    )
                )
            ]["text"].values
        )

        # Store image description
        final_images[matched_imageno]["image_description"] = image_metadata_df.iloc[
            indexvalue
        ]["img_desc"]

    return final_images


def get_similar_text_from_query(
    query: str,
    text_metadata_df: pd.DataFrame,
    column_name: str = "",
    top_n: int = 3,
    chunk_text: bool = True,
    print_citation: bool = False,
) -> Dict[int, Dict[str, Any]]:
    """
    Finds the top N most similar text passages from a metadata DataFrame based on a text query.

    Args:
        query: The text query used for finding similar passages.
        text_metadata_df: A Pandas DataFrame containing the text metadata to search.
        column_name: The column name in the text_metadata_df containing the text embeddings or text itself.
        top_n: The number of most similar text passages to return.
        embedding_size: The dimensionality of the text embeddings (only used if text embeddings are stored in the column specified by `column_name`).
        chunk_text: Whether to return individual text chunks (True) or the entire page text (False).
        print_citation: Whether to immediately print formatted citations for the matched text passages (True) or just return the dictionary (False).

    Returns:
        A dictionary containing information about the top N most similar text passages, including cosine scores, page numbers, chunk numbers (optional), and chunk text or page text (depending on `chunk_text`).

    Raises:
        KeyError: If the specified `column_name` is not present in the `text_metadata_df`.
    """

    if column_name not in text_metadata_df.columns:
        raise KeyError(f"Column '{column_name}' not found in the 'text_metadata_df'")

    query_vector = get_user_query_text_embeddings(query)


    # Calculate cosine similarity between query text and metadata text
    cosine_scores = text_metadata_df.apply(
        lambda row: get_cosine_score(
            row,
            column_name,
            query_vector,
        ),
        axis=1,
    )

    # Get top N cosine scores and their indices
    top_n_indices = cosine_scores.nlargest(top_n).index.tolist()
    top_n_scores = cosine_scores.nlargest(top_n).values.tolist()

    # Create a dictionary to store matched text and their information
    final_text: Dict[int, Dict[str, Any]] = {}

    for matched_textno, index in enumerate(top_n_indices):
        # Create a sub-dictionary for each matched text
        final_text[matched_textno] = {}

        #Code subIndex  Desc Longer Desc

        # Store page number
        final_text[matched_textno]["clip_name"] = text_metadata_df.iloc[index][
            "clip_name"
        ]

        # Store page number
        final_text[matched_textno]["description	"] = text_metadata_df.iloc[index][
            "description"
        ]

        # Store cosine score
        final_text[matched_textno]["cosine_score"] = top_n_scores[matched_textno]


    return final_text


def display_images(
    images: Iterable[Union[str, PIL.Image.Image]], resize_ratio: float = 0.5
) -> None:
    """
    Displays a series of images provided as paths or PIL Image objects.

    Args:
        images: An iterable of image paths or PIL Image objects.
        resize_ratio: The factor by which to resize each image (default 0.5).

    Returns:
        None (displays images using IPython or Jupyter notebook).
    """

    # Convert paths to PIL images if necessary
    pil_images = []
    for image in images:
        if isinstance(image, str):
            pil_images.append(PIL.Image.open(image))
        else:
            pil_images.append(image)

    # Resize and display each image
    for img in pil_images:
        original_width, original_height = img.size
        new_width = int(original_width * resize_ratio)
        new_height = int(original_height * resize_ratio)
        resized_img = img.resize((new_width, new_height))
        display(resized_img)
        print("\n")

## Cut the video into subclips of specified duration

In [10]:
import moviepy.editor as mpe
import pandas as pd
import math

# Load the video file
clip = mpe.VideoFileClip(downloaded_video)

# first result is retrieved because a single video was processed
shots_list = []

shots_df = pd.read_csv('shots.csv')

# e.g. Cut into 60s clips (Change clip_duration to set the desired duration of subclips)
clip_duration = 60
clip_no = math.ceil(clip.duration/clip_duration)
clip_no = int(clip_no)

unnamed_id_list = []
start_time_list = []
end_time_list = []
clip_name_list = []
frame_name_list = []

for i in range(clip_no):
    start_time = i * clip_duration
    end_time = min(start_time + clip_duration, clip.duration)

    text = ""
    clip_name = "content/clips/clip_"+str(i)+".mp4"
    clip.subclip(start_time, end_time).write_videofile(clip_name, logger=None)
    time = (start_time  + end_time)/2
    print(f"{clip_name} has been processed")
   # saving a frame at 2 second
    frame_name = "content/frames/frame_"+str(i)+".png"
    clip.save_frame(frame_name, t = float(time))

    unnamed_id_list.append(i-0)
    start_time_list.append(start_time)
    end_time_list.append(end_time)
    clip_name_list.append(clip_name)
    frame_name_list.append(frame_name)
      
print(f"\n{clip_no} clips have been processed.")
        
shots_df['id'] = unnamed_id_list
shots_df['start_time'] = start_time_list
shots_df['end_time'] = end_time_list
shots_df['clip_name'] = clip_name_list
shots_df['frame_name'] = frame_name_list

content/clips/clip_0.mp4 has been processed
content/clips/clip_1.mp4 has been processed
content/clips/clip_2.mp4 has been processed
content/clips/clip_3.mp4 has been processed
content/clips/clip_4.mp4 has been processed
content/clips/clip_5.mp4 has been processed
content/clips/clip_6.mp4 has been processed
content/clips/clip_7.mp4 has been processed
content/clips/clip_8.mp4 has been processed
content/clips/clip_9.mp4 has been processed

10 clips have been processed.


## Embeddings

### Load the Video

In [11]:
import base64

from vertexai.preview.generative_models import GenerativeModel, Part
import base64


text = ""
with open("content/clips/clip_0.mp4", "rb") as videoFile:
    text = base64.b64encode(videoFile.read())
    print(text)

video1 = Part.from_data(data=base64.b64decode(text), mime_type="video/mp4")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Helper Functions to generate embeddings

In [12]:
import base64
import vertexai
import vertexai.preview.generative_models as generative_models


#Generate video description using AI Model
#Model used is customisable (default = gemini-1.0-pro)

def generate(video):
    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = GenerativeModel("gemini-1.5-pro-001")
    responses = model.generate_content(
    ["""You are provided with a short video. First, watch the whole video in full. Then, 
    describe the contents of the video in chronological order. Register everything that is said in the video as well.
    Then, provide detailed descriptions of each subject present in the video,
    include their appearance, emotion, actions if possible. Include any names if mentioned.
    Go into as much detail as possible.""", video1],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0,
        "top_p": 0.4,
        "top_k": 32
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=False,
    )

  #print(responses.text, end="")
    return responses.text

def generateText(video):
    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = GenerativeModel("gemini-1.5-pro-001")
    responses = model.generate_content(
    ["""You are provided with a short video. Register everything visible in chronological order.
    Do not include audio. Only include what you can see visually.""", video1],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0,
        "top_p": 0.4,
        "top_k": 32
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=False,
    )

  #print(responses.text, end="")
    return responses.text



def generateSpeech(video):
    
    
    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = GenerativeModel("gemini-1.5-pro-001")
    responses = model.generate_content(
    ["""You are provided with a short video. Watch the video in full, and listen to the audio.
    Register everything you hear in chronological order. Convert everything into text, word for word.""", video1],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0,
        "top_p": 0.4,
        "top_k": 32
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=False,
    )

  #print(responses.text, end="")
    return responses.text

def generateObjects(video):
    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = GenerativeModel("gemini-1.5-pro-001")
    responses = model.generate_content(
    ["""You are provided with a short video. Describe all the different people or animals that appear throughout the video. Associate them with
    a name if mentioned.
    Include their appearance, emotions, actions if possible. Do so in great detail.""", video1],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0,
        "top_p": 0.4,
        "top_k": 32
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=False,
    )

  #print(responses.text, end="")
    return responses.text

generateSpeech(video1)

"At Google we are fully in our Gemini era Today all of our 2 billion user products use Gemini Gemini 1.5 Pro is available today in Workspace Labs Let's see how this comes to life with Google Workspace People are always searching their emails in Gmail We're working to make it much more powerful with Gemini Now you can ask Gemini to summarize all recent emails from the school Maybe you were traveling this week and you couldn't make the PTA meeting The recording of the meeting is an hour long If it's from Google Meet you can ask Gemini to give you the highlights People love using photos to search across their life With Gemini we're making that a whole lot easier And Ask Photos can also help you search your memories in a deeper way For example you might be reminiscing \n"

In [13]:
associated_desc=[]
associated_speech=[]
associated_text=[]
associated_object=[]



for i in range(clip_no):
    clip_name = f"clip_{i}"
    text = ""
    with open(f"content/clips/{clip_name}.mp4", "rb") as videoFile:
        text = base64.b64encode(videoFile.read())

        video1 = Part.from_data(data=base64.b64decode(text), mime_type="video/mp4")
        desc = generate(video1)
        associated_desc.append(desc)
        speechDesc = generateSpeech(video1)
        associated_speech.append(speechDesc)
        textDesc = generateText(video1)
        associated_text.append(textDesc)
        objDesc = generateObjects(video1)
        associated_object.append(objDesc)
        
        
for i in range(clip_no):
    clip_name = f"clip_{i}"
    text = ""
    with open(f"content/clips/{clip_name}.mp4", "rb") as videoFile:
        text = base64.b64encode(videoFile.read())

        video1 = Part.from_data(data=base64.b64decode(text), mime_type="video/mp4")
        

shots_df['description'] = associated_desc
shots_df['associated_speech'] = associated_speech
shots_df['associated_text'] = associated_text
shots_df['associated_object'] = associated_object
shots_df.head(5)

,Id,start_time,end_time,clip_name,frame_name,associated_text,associated_speech,associated_object,description,id
0,NaN,0,60.0,content/clips/clip_0.mp4,content/frames/frame_0.png,The video starts with a white screen and a rec...,At Google we are fully in our Gemini era Today...,"The video features Sundar Pichai, CEO of Googl...",The video starts with the Google I/O logo anim...,0
1,NaN,60,120.0,content/clips/clip_1.mp4,content/frames/frame_1.png,The video starts with a man with glasses stand...,about your daughter Lucia's early milestones Y...,"The video features a single speaker, a man who...",The video starts with a man standing on a stag...,1
2,NaN,120,180.0,content/clips/clip_2.mp4,content/frames/frame_2.png,- A man with glasses and a gray shirt is stand...,the opportunities we see with AI agents I thin...,The video features two people.\n\nThe first pe...,"The video starts with Sundar Pichai, CEO of Go...",2
3,NaN,180,240.0,content/clips/clip_3.mp4,content/frames/frame_3.png,The video starts with a person holding a phone...,vector IV Do you remember where you saw my gla...,The video starts with a pair of hands holding ...,The video starts with a person holding a smart...,3
4,NaN,240,300.0,content/clips/clip_4.mp4,content/frames/frame_4.png,The video starts with a black screen and five ...,video prompts It can capture the details of yo...,The video starts with a fast-tracking shot thr...,"The video starts with a montage of images, inc...",4


In [14]:
from vertexai.language_models import TextEmbeddingModel
text_embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

In [15]:
embeddingsList = []
for i in range(clip_no):
    string = shots_df['description'][i] + "\n" + shots_df['associated_speech'][i] + "\n" + shots_df['associated_text'][i] + "\n" +  shots_df['associated_object'][i]
    embeddings = text_embedding_model.get_embeddings([string])
    text_embedding = [embedding.values for embedding in embeddings][0]
    embeddingsList.append(text_embedding)
shots_df["embedding"] = embeddingsList

### Preview Dataframe 

In [16]:
shots_df.head(5)

,Id,start_time,end_time,clip_name,frame_name,associated_text,associated_speech,associated_object,description,id,embedding
0,NaN,0,60.0,content/clips/clip_0.mp4,content/frames/frame_0.png,The video starts with a white screen and a rec...,At Google we are fully in our Gemini era Today...,"The video features Sundar Pichai, CEO of Googl...",The video starts with the Google I/O logo anim...,0,"[0.02301250398159027, 0.01784183457493782, -0...."
1,NaN,60,120.0,content/clips/clip_1.mp4,content/frames/frame_1.png,The video starts with a man with glasses stand...,about your daughter Lucia's early milestones Y...,"The video features a single speaker, a man who...",The video starts with a man standing on a stag...,1,"[0.018422599881887436, 0.025941209867596626, -..."
2,NaN,120,180.0,content/clips/clip_2.mp4,content/frames/frame_2.png,- A man with glasses and a gray shirt is stand...,the opportunities we see with AI agents I thin...,The video features two people.\n\nThe first pe...,"The video starts with Sundar Pichai, CEO of Go...",2,"[-0.0005373490275815129, -0.004592179320752621..."
3,NaN,180,240.0,content/clips/clip_3.mp4,content/frames/frame_3.png,The video starts with a person holding a phone...,vector IV Do you remember where you saw my gla...,The video starts with a pair of hands holding ...,The video starts with a person holding a smart...,3,"[0.028424551710486412, -0.034521788358688354, ..."
4,NaN,240,300.0,content/clips/clip_4.mp4,content/frames/frame_4.png,The video starts with a black screen and five ...,video prompts It can capture the details of yo...,The video starts with a fast-tracking shot thr...,"The video starts with a montage of images, inc...",4,"[0.01569233275949955, -0.006282717455178499, -..."


In [17]:
shots_df.to_csv('content/shots.csv')
# Copy the file to our new bucket.
!gsutil cp content/shots.csv {BUCKET_URI}/shots.csv

Copying file://content/shots.csv [Content-Type=text/csv]...
/ [1 files][235.8 KiB/235.8 KiB]                                                
Operation completed over 1 objects/235.8 KiB.                                    


### Convert the Dataframe into a JSON file, which will be uploaded into Vector Search Index

In [18]:
from datetime import datetime


BUCKET_URI_ME=f"{BUCKET_URI}/embeddings/"
LOCATION = 'us-central1'

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

jsonl_string = shots_df[['id','start_time','end_time','clip_name', 'frame_name', "description", "embedding"]].to_json(orient="records", lines=True)
with open(f"./videodata.json", "w") as f:
    f.write(jsonl_string)

In [19]:
! gsutil cp videodata.json {BUCKET_URI_ME}

Copying file://videodata.json [Content-Type=application/json]...
/ [1 files][133.1 KiB/133.1 KiB]                                                
Operation completed over 1 objects/133.1 KiB.                                    


## Creating Vector Search Index

In [20]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-feature-index-{UID}",
    contents_delta_uri=BUCKET_URI_ME,
    dimensions=768,
    approximate_neighbors_count=10,
    project = PROJECT_ID
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/255766800726/locations/us-central1/indexes/8368299436119425024/operations/2987475519047467008
MatchingEngineIndex created. Resource name: projects/255766800726/locations/us-central1/indexes/8368299436119425024
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/255766800726/locations/us-central1/indexes/8368299436119425024')


### Create Index Endpoint and deploy the Index
To use the Index, you need to create an Index Endpoint. It works as a server instance accepting query requests for your Index.


In [21]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"darryl-testing-lh-{UID}", public_endpoint_enabled=True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136/operations/9076342215252377600
MatchingEngineIndexEndpoint created. Resource name: projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136')


In [ ]:
DEPLOYED_INDEX_ID = f"darryl_testing_lh_{UID}"
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136/operations/5333287974954074112
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136


resource name: projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136

#### Go to your Vertex AI console and check that the index is CREATED successfully 

### Get an existing Index

In [36]:
%pip install tqdm -q
import tqdm
import time


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
REGION = LOCATION = "us-central1"

aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [47]:
my_index_endpoint_id= '2987475519047467008' #'8368299436119425024'
PROJECT_ID

'my-project-0004-346516'

In [48]:
my_index_name = my_index._gca_resource.name
my_index_display_name = my_index.display_name
my_index_id = my_index.name.split('/')[-1]

my_index_endpoint_name = my_index_endpoint._gca_resource.name
my_index_endpoint_display_name = my_index_endpoint.display_name
my_index_endpoint_id = my_index_endpoint.name.split('/')[-1]
my_index_endpoint_public_domain = my_index_endpoint.public_endpoint_domain_name

my_index = aiplatform.MatchingEngineIndex(my_index_name)

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)

In [49]:
from google.cloud import aiplatform_v1

# Set variables for the current deployed index.
API_ENDPOINT=my_index_endpoint_public_domain
INDEX_ENDPOINT=my_index_endpoint_name

indexendpoint_id=UNIQUE_PREFIX

DEPLOYED_INDEX_ID="darryl_testing_lh_" + indexendpoint_id
neighbor_count = 3

print(API_ENDPOINT)
print(INDEX_ENDPOINT)
print(DEPLOYED_INDEX_ID)

631876179.us-central1-255766800726.vdb.vertexai.goog
projects/255766800726/locations/us-central1/indexEndpoints/8281781065152987136
darryl_testing_lh_pytorch112kagglewbi


In [50]:
# %pip install -q langchain_google_vertexai

In [51]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="textembedding-gecko@003", model_name="textembedding-gecko@003")

text_embedding_model = embeddings #TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

### Querying using Vector Search

In [52]:
#Set your query
query = "Give me the clips where there are animals present"

#Set the number of results you want
neighbor_count = 3

test_embeddings = embeddings.embed_query(query)

In [53]:
# Configure Vector Search client
client_options = {
  "api_endpoint": API_ENDPOINT
}
vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)
# Build FindNeighborsRequest object
datapoint = aiplatform_v1.IndexDatapoint(
  feature_vector=test_embeddings
)

query = aiplatform_v1.FindNeighborsRequest.Query(
  datapoint=datapoint,
  # The number of nearest neighbors to be retrieved
  neighbor_count=neighbor_count
)

request = aiplatform_v1.FindNeighborsRequest(
  index_endpoint=INDEX_ENDPOINT,
  deployed_index_id=DEPLOYED_INDEX_ID,
  # Request can have multiple queries
  queries=[query],
  return_full_datapoint=False,
)

# Execute the request
response = vector_search_client.find_neighbors(request)

df_new = pd.DataFrame()
print('neighbor_count', neighbor_count)

shots_df['distance'] = None

for i in range(0,neighbor_count):
    x=response.nearest_neighbors[0]
    
    df_match = shots_df.loc[shots_df['id'] == int(x.neighbors[i].datapoint.datapoint_id) ]
    df_match['distance'] = x.neighbors[i].distance

    # Append the matching rows to the new DataFrame
    df_new = pd.concat([df_new, df_match])
    

# Print the new DataFrame
df_sorted = df_new.sort_values(by="distance", ascending=True)
print(display(df_sorted))

#Export DataFrame to CSV file for reference
df_new.to_csv("results.csv")

neighbor_count 3


,Id,start_time,end_time,clip_name,frame_name,associated_text,associated_speech,associated_object,description,id,embedding,distance
2,NaN,120,180.0,content/clips/clip_2.mp4,content/frames/frame_2.png,- A man with glasses and a gray shirt is stand...,the opportunities we see with AI agents I thin...,The video features two people.\n\nThe first pe...,"The video starts with Sundar Pichai, CEO of Go...",2,"[-0.0005373490275815129, -0.004592179320752621...",0.603239
4,NaN,240,300.0,content/clips/clip_4.mp4,content/frames/frame_4.png,The video starts with a black screen and five ...,video prompts It can capture the details of yo...,The video starts with a fast-tracking shot thr...,"The video starts with a montage of images, inc...",4,"[0.01569233275949955, -0.006282717455178499, -...",0.606726
3,NaN,180,240.0,content/clips/clip_3.mp4,content/frames/frame_3.png,The video starts with a person holding a phone...,vector IV Do you remember where you saw my gla...,The video starts with a pair of hands holding ...,The video starts with a person holding a smart...,3,"[0.028424551710486412, -0.034521788358688354, ...",0.637677


None


In [54]:
clipNames = []

for i in df_sorted['clip_name']:
    clipNames.append(i)
    
#Display the clips     
print(clipNames)

['content/clips/clip_2.mp4', 'content/clips/clip_4.mp4', 'content/clips/clip_3.mp4']


### Display Clips

In [55]:
import IPython

for i in clipNames:
    IPython.display.display(IPython.display.Video(i, width=600))

In [56]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

FOLDER_NAME="."

dataset_id = "darryl_video_rag_demo_lh"
table_id = "video_embeddings"
LOCATION="us-central1"
full_table_id = '.'.join([PROJECT_ID,dataset_id,table_id])
file_path = "embeddings_videodata.json"


from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
from datetime import datetime

In [59]:
def create_bigquery_dataset():
    dataset = client.dataset(dataset_id)
    dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

def create_bigquery_table():    
    # Define the schema for the table
    schema = [
        bigquery.SchemaField("id", "STRING"),
        bigquery.SchemaField("start_time", "STRING"),
        bigquery.SchemaField("end_time", "STRING"),
        bigquery.SchemaField("clip_name", "STRING"),
        bigquery.SchemaField("frame_name", "STRING"),
        bigquery.SchemaField("description", "STRING"),
        bigquery.SchemaField("embedding", "FLOAT", mode="REPEATED")
    ]
    
    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Define the table object
    table = bigquery.Table(table_ref, schema=schema)
    
    # Create the table in BigQuery
    try:
        client.create_table(table)
        print("Table created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

def load_data_into_bigquery_table():
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON, autodetect=False)
    
    table_ref = client.dataset(dataset_id).table(table_id)
    
    with open(file_path, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    
    job.result()  # Waits for the job to complete.
    
    table = client.get_table(table_ref)  # Make an API request.
    print("Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_ref))
    
def read_bq_table_dataframe():  
    table_ref = client.dataset(dataset_id).table(table_id)
    dataframe = client.list_rows(table_ref).to_dataframe(create_bqstorage_client=True)
    
    return dataframe

In [61]:
# Call the function to create the table
# create_bigquery_dataset()
create_bigquery_table()
load_data_into_bigquery_table()

Error creating table: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/my-project-0004-346516/datasets/darryl_video_rag_demo_lh/tables?prettyPrint=false: Already Exists: Table my-project-0004-346516:darryl_video_rag_demo_lh.video_embeddings
Loaded 10 rows and 7 columns to my-project-0004-346516.darryl_video_rag_demo_lh.video_embeddings


In [62]:
dataframe = read_bq_table_dataframe()
dataframe

,id,start_time,end_time,clip_name,frame_name,description,embedding
0,0,0,60.0,content/clips/clip_0.mp4,content/frames/frame_0.png,The video starts with the Google I/O logo anim...,"[0.023012504, 0.0178418346, -0.0278227255, 0.0..."
1,1,60,120.0,content/clips/clip_1.mp4,content/frames/frame_1.png,The video starts with a man standing on a stag...,"[0.0184225999, 0.0259412099, -0.0144431163, 0...."
2,2,120,180.0,content/clips/clip_2.mp4,content/frames/frame_2.png,"The video starts with Sundar Pichai, CEO of Go...","[-0.000537349, -0.0045921793, -0.0360602662, -..."
3,3,180,240.0,content/clips/clip_3.mp4,content/frames/frame_3.png,The video starts with a person holding a smart...,"[0.0284245517, -0.0345217884, -0.029420957, 0...."
4,4,240,300.0,content/clips/clip_4.mp4,content/frames/frame_4.png,"The video starts with a montage of images, inc...","[0.0156923328, -0.0062827175, -0.0356400497, 0..."
5,5,300,360.0,content/clips/clip_5.mp4,content/frames/frame_5.png,The video starts with a woman with short curly...,"[0.0248258356, -0.0402916409, -0.0059038997, 0..."
6,6,360,420.0,content/clips/clip_6.mp4,content/frames/frame_6.png,The video starts with a woman standing on a st...,"[0.0603798144, -0.0208670031, -0.0023623984, -..."
7,7,420,480.0,content/clips/clip_7.mp4,content/frames/frame_7.png,The video starts with a woman standing on a st...,"[0.0107160276, 0.0019427086, -0.0039142366, 0...."
8,8,480,540.0,content/clips/clip_8.mp4,content/frames/frame_8.png,"The video is about Google's new AI model, Gemi...","[-0.0237723123, -0.0174860042, -0.0038281605, ..."
9,9,540,598.05,content/clips/clip_9.mp4,content/frames/frame_9.png,"The video starts with a man with glasses, wear...","[0.003135087, -0.0236313213, -0.0164693557, 0...."


## Querying using Gemini Model instead of Vector Search

### Defining Functions for Gemini

In [63]:
import os
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.preview.language_models import TextGenerationModel

def generate(input_prompt):
    model = GenerativeModel("gemini-ultra")
    responses = model.generate_content(
        input_prompt ,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1,
        "top_k": 32
    },
        safety_settings=[],
        stream=True,
    )
    
    all_response  = []
    
    for response in responses:
        # print(response.text, end="")
        all_response.append(response.text)
    
    return(" ".join(all_response))
    

def generate_pro(input_prompt):
    model = GenerativeModel("gemini-pro")
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1
    },stream=True,)
    
    all_response  = []
    
    for response in responses:
        all_response.append(response.text)

    return(" ".join(all_response))


### Query using Gemini

In [64]:
#Insert with your own query
query = "In which clip is there a dog?"

In [65]:
import pandas as pd

def combine_column_to_string(df, column_name):

    column_values = df[column_name].tolist()
    combined_string = ', '.join(column_values)
    return combined_string

In [66]:
import pandas as pd
import re

System_Prompts = """ You are an expert in screening videos and understanding the context and contents of the video.
Only answer based on the contents of the video provided here: 
"""

Question_Prompts = """ -- Based on video provided, answer the following query as accurately as possible:
"""

videoDesc = combine_column_to_string(shots_df,'description') + combine_column_to_string(shots_df,'associated_text') + combine_column_to_string(shots_df,'associated_speech') + combine_column_to_string(shots_df,'associated_object')

combined_prompt = System_Prompts + ' ' + videoDesc + ' ' + Question_Prompts + query


print("Your prompt: \n" + combined_prompt)


Your prompt: 
 You are an expert in screening videos and understanding the context and contents of the video.
Only answer based on the contents of the video provided here: 
 The video starts with the Google I/O logo animation, transitioning to a man walking on a stage in front of a large audience.

The man on the stage says: "At Google, we are fully in our Gemini era. Today, all of our 2 billion user products use Gemini. Gemini 1.5 Pro is available today in Workspace Labs. Let's see how this comes to life with Google Workspace. People are always searching their emails in Gmail. We are working to make it much more powerful with Gemini. Now, you can ask Gemini to summarize all recent emails from the school. Maybe you were traveling this week and you couldn't make the PTA meeting. The recording of the meeting is an hour long. If it's from Google Meet, you can ask Gemini to give you the highlights. People love using photos to search across their life. With Gemini, we're making that a whole

In [67]:
print("Response from AI model: \n")
print(generate_pro(combined_prompt))

Response from AI model: 

The  dog appears in the clip with the woman wearing a denim jacket and holding a stuffed tiger toy.  
 


## The End